In [2]:
import pandas as pd
import json
import getpass
import requests
import datetime
import os


In [3]:
WB_TOKEN = os.getenv('WB_TOKEN')
headers = {'Authorization': WB_TOKEN}

In [ ]:
subj_chars_url = 'https://content-api.wildberries.ru/content/v2/object/charcs/3979'
resp = requests.get(subj_chars_url, headers=headers)


'{"data":[{"charcID":59611,"subjectName":"Коробки картонные","subjectID":3979,"name":"Назначение подарка","required":false,"unitName":"","maxCount":3,"popular":false,"charcType":1},{"charcID":88952,"subjectName":"Коробки картонные","subjectID":3979,"name":"Вес товара с упаковкой (г)","required":false,"unitName":"г","maxCount":0,"popular":false,"charcType":4},{"charcID":89008,"subjectName":"Коробки картонные","subjectID":3979,"name":"Вес товара без упаковки (г)","required":false,"unitName":"г","maxCount":0,"popular":false,"charcType":4},{"charcID":90630,"subjectName":"Коробки картонные","subjectID":3979,"name":"Высота предмета","required":false,"unitName":"см","maxCount":0,"popular":false,"charcType":4},{"charcID":90652,"subjectName":"Коробки картонные","subjectID":3979,"name":"Глубина предмета","required":false,"unitName":"см","maxCount":0,"popular":false,"charcType":4},{"charcID":90673,"subjectName":"Коробки картонные","subjectID":3979,"name":"Ширина предмета","required":false,"unitNa

In [5]:
d = json.loads( resp.text)

In [12]:
d

{'data': [{'charcID': 59611,
   'subjectName': 'Коробки картонные',
   'subjectID': 3979,
   'name': 'Назначение подарка',
   'required': False,
   'unitName': '',
   'maxCount': 3,
   'popular': False,
   'charcType': 1},
  {'charcID': 88952,
   'subjectName': 'Коробки картонные',
   'subjectID': 3979,
   'name': 'Вес товара с упаковкой (г)',
   'required': False,
   'unitName': 'г',
   'maxCount': 0,
   'popular': False,
   'charcType': 4},
  {'charcID': 89008,
   'subjectName': 'Коробки картонные',
   'subjectID': 3979,
   'name': 'Вес товара без упаковки (г)',
   'required': False,
   'unitName': 'г',
   'maxCount': 0,
   'popular': False,
   'charcType': 4},
  {'charcID': 90630,
   'subjectName': 'Коробки картонные',
   'subjectID': 3979,
   'name': 'Высота предмета',
   'required': False,
   'unitName': 'см',
   'maxCount': 0,
   'popular': False,
   'charcType': 4},
  {'charcID': 90652,
   'subjectName': 'Коробки картонные',
   'subjectID': 3979,
   'name': 'Глубина предмета',
 

In [14]:
names = [n['name'] + "->" + str(n['required']) for n in d['data']]

In [28]:
import os
import re
import sqlite3

def gather_products_from_folder_and_db(db_path, images_folder):
    """
    1. Берёт все файлы формата 'product_id-qty.jpg' из заданной папки.
    2. Извлекает из имени файла product_id и количество (qty).
    3. Достаёт данные из таблицы products по product_id.
    4. Формирует структуру для Wildberries (subjectID, variants и т.д.).
    5. Возвращает список объектов, готовый для отправки в API.
    """
    
    # Регулярное выражение для поиска product_id-qty.jpg
    pattern = re.compile(r'^(\d+)-(\d+)\.jpg$', re.IGNORECASE)
    
    # Соединяемся с БД
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Список, куда будем складывать товары
    products_list = []
    
    # Обходим все файлы в папке
    for filename in os.listdir(images_folder):
        match = pattern.match(filename)
        if not match:
            continue
        
        product_id, quantity_str = match.groups()
        
        # Пытаемся получить данные о товаре по product_id
        cursor.execute("SELECT * FROM products WHERE product_id = ?", (product_id,))
        row = cursor.fetchone()
        
        # Если в БД нет записи — пропускаем
        if not row:
            continue
        
        # Раскладываем строки по названным полям (см. структуру CREATE TABLE products)
        (
            row_id,
            db_product_id,
            name,
            price_str,
            material,
            length_mm,
            width_mm,
            height_mm,
            volume_l,
            cardboard_brand,
            cardboard_type,
            color,
            has_handles,
            box_shape,
            weight_g,
            transport_volume_m3,
            trade_mark,
            manufacturer_country
        ) = row
        
        # Конвертируем некоторые значения в см (из мм)
        # Высота предмета
        height_cm = None
        if height_mm:
            try:
                height_cm = round(float(height_mm) / 10, 2)  # 'см', округлим до 2 знаков
            except:
                pass
        
        # Глубина (пусть будет length_mm)
        depth_cm = None
        if length_mm:
            try:
                depth_cm = round(float(length_mm) / 10, 2)
            except:
                pass
        
        # Ширина (width_mm)
        width_cm = None
        if width_mm:
            try:
                width_cm = round(float(width_mm) / 10, 2)
            except:
                pass
        
        # Цена (из строки уберём буквы, оставим только число)
        # Например, "30.35 p" -> 30.35 -> 30 (округлённая) или в копейках как целое число
        # price_val = 0
        # if price_str:
        #     try:
        #         # Удалим всё кроме цифр и точки
        #         price_num = re.sub(r'[^0-9.,]', '', price_str).replace(',', '.')
        #         price_val = int(float(price_num))
        #     except:
        #         pass
        
        # Характеристики. Собираем массив characteristics в зависимости от непустых полей.
        characteristics = []
        
        # Количество предметов в упаковке (charcID=179792) -> из quantity_str
        if quantity_str:
            characteristics.append({"id": 179792, "value": [quantity_str]})
        
        # Вес товара с упаковкой (г) (charcID=88952) -> из weight_g
        if weight_g:
            characteristics.append({"id": 88952, "value": int(float(weight_g)*int(quantity_str))})
        
        # Цвет (charcID=14177449)
        if color:
            characteristics.append({"id": 14177449, "value": ["крафт"]})
        
        # Страна производства (charcID=14177451)
        if manufacturer_country:
            characteristics.append({"id": 14177451, "value": [manufacturer_country]})
        
        # Марка картона (charcID=15002205)
        if cardboard_brand:
            characteristics.append({"id": 15002205, "value": [cardboard_brand]})
        
        # Форма коробки (charcID=15000432)
        if box_shape:
            characteristics.append({"id": 15000432, "value": [box_shape]})
        
        # Объем товара (л) (charcID=15000783)
        # if volume_l:
        #     try:
        #         volume_val = float(volume_l)
        #         characteristics.append({"id": 15000783, "value": volume_val})
        #     except:
        #         pass
        
        # Высота предмета (charcID=90630)
        if height_cm is not None:
            characteristics.append({"id": 90630, "value": height_cm})
        
        # Глубина предмета (charcID=90652)
        if depth_cm is not None:
            characteristics.append({"id": 90652, "value": depth_cm})
        
        # Ширина предмета (charcID=90673)
        if width_cm is not None:
            characteristics.append({"id": 90673, "value": width_cm})
        
        # Формируем title: к наименованию добавляем, например, "(10 шт.)"
        final_title = f"{name} {quantity_str} шт."
        
        # vendorCode: box_{product_id}_{количество}
        vendor_code = f"box_{product_id}_{quantity_str}"
        
        # dimensions — размеры упаковки для Wildberries (указываем в сантиметрах)
        # Если нужно указать точные габариты, можно взять из length_cm, width_cm, height_cm
        # Здесь, как пример, возьмём их, округляя до int
        dimensions = {
            # "length": int(depth_cm) if depth_cm else 0,
            "length": int((height_cm*2) + (depth_cm * 2)) if height_cm else 0,
            "width": int((width_cm*2) +  (depth_cm * 2)) if width_cm else 0,
            "height": int(int(quantity_str) * 0.3) if height_cm else 0
        }
        
        # Собираем "variants". В sizes можно указать одну "размерную" сущность
        # c ценой price_val (или более сложную логику, если нужно).
        variant = {
            "vendorCode": vendor_code,
            "title": final_title,
            # "description": f"Изготовлено из материала: {material}.",
            # "brand": trade_mark if trade_mark else "NoName",
            "dimensions": dimensions,
            "characteristics": characteristics,
            # "sizes": [
            #     {
            #         "techSize": "OneSize",
            #         "wbSize": "1",
            #         # "price": price_val,
            #         # "skus": [
            #         #     f"{product_id}-{quantity_str}"
            #         # ]
            #     }
            # ]
        }
        
        # subjectID 3979 — "Коробки картонные"
        product_object = {
            "subjectID": 3979,
            "variants": [variant]
        }
        
        # Добавляем в итоговый список
        products_list.append(product_object)
    
    # Закрываем соединение
    conn.close()
    
    return products_list




In [29]:

db_file = "products.db"  # путь к вашей базе данных
folder_path = "images_900_1200"

result = gather_products_from_folder_and_db(db_file, folder_path)
for item in result:
    print(item)
# Здесь item — уже готовая структура для Wildberries API.
# Дальше можно вызывать свой метод create_product_cards(result), как в предыдущем примере.


{'subjectID': 3979, 'variants': [{'vendorCode': 'box_9604580_50', 'title': 'Коробка №204/1 (премиум), беленая, 135*85*55 мм 50 шт.', 'dimensions': {'length': 38, 'width': 44, 'height': 15}, 'characteristics': [{'id': 179792, 'value': ['50']}, {'id': 88952, 'value': 1300}, {'id': 14177449, 'value': ['крафт']}, {'id': 14177451, 'value': ['Россия']}, {'id': 15002205, 'value': ['Е-22']}, {'id': 15000432, 'value': ['Плоская']}, {'id': 90630, 'value': 5.5}, {'id': 90652, 'value': 13.5}, {'id': 90673, 'value': 8.5}]}]}
{'subjectID': 3979, 'variants': [{'vendorCode': 'box_9601645_30', 'title': 'Картонная коробка для бутылки №95, 100*100*310 мм 30 шт.', 'dimensions': {'length': 82, 'width': 40, 'height': 9}, 'characteristics': [{'id': 179792, 'value': ['30']}, {'id': 88952, 'value': 2400}, {'id': 14177449, 'value': ['крафт']}, {'id': 14177451, 'value': ['Россия']}, {'id': 15002205, 'value': ['Т-24 В']}, {'id': 15000432, 'value': ['Высокая']}, {'id': 90630, 'value': 31.0}, {'id': 90652, 'value':

In [30]:
API_URL = "https://content-api.wildberries.ru/content/v2/cards/upload"



def create_product_cards(products):
  
    try:
        response = requests.post(API_URL, json=products, headers=headers)
        response_data = response.json()
        
        if response.status_code == 200 and not response_data.get("error", True):
            print("Карточки товаров успешно созданы.")
        else:
            print(f"Ошибка: {response_data.get('errorText', 'Неизвестная ошибка')}")
            print("Дополнительные ошибки:", response_data.get("additionalErrors", {}))
        
        return response_data
    except requests.RequestException as e:
        print(f"Ошибка при выполнении запроса: {e}")
        return None

In [33]:
resp = create_product_cards(result[:90])

Карточки товаров успешно созданы.
